Age
Total Spend
Items Purchased
Average Rating
Discount Applied
Days Since Last Purchase
Satisfaction Level
City
Gender
Membership Type

In [1]:
import os
import glob
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\user\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [15]:
main_dir = 'data/processed/'
file_name = 'cluster6_Miami.csv'
file_path = os.path.join(main_dir, file_name)

In [16]:
df = pd.read_csv(file_path)
df.head()

,Age,Total Spend,Items Purchased,Average Rating,Discount Applied,Days Since Last Purchase,Satisfaction Level,Miami_Young_Adults,Miami_Middle-aged_Adults,Gender_Female,Gender_Male,City_Miami,Membership Type_Silver,Cluster
0,0.058824,0.279095,0.428571,0.526316,1.0,0.851852,0.666667,1.0,0.0,0.0,1.0,1.0,1.0,5
1,0.352941,0.251961,0.285714,0.421053,1.0,0.462963,0.666667,0.0,1.0,0.0,1.0,1.0,1.0,5
2,0.000000,0.261246,0.357143,0.368421,1.0,0.722222,0.666667,1.0,0.0,0.0,1.0,1.0,1.0,5
3,0.058824,0.270080,0.428571,0.578947,1.0,0.833333,0.666667,1.0,0.0,0.0,1.0,1.0,1.0,5
4,0.352941,0.233931,0.214286,0.421053,1.0,0.444444,0.666667,0.0,1.0,0.0,1.0,1.0,1.0,5


In [17]:
# Step 1: Select the features for the recommendation system
features = ['Age', 'Total Spend', 'Items Purchased', 'Average Rating', 'Discount Applied',
            'Days Since Last Purchase', 'Satisfaction Level', 'City_Miami', 'Gender_Female',
            'Membership Type_Silver']

In [18]:
# Step 2: Extract the relevant features from the DataFrame
df_features = df[features]

In [19]:
# Step 3: Compute cosine similarity
cosine_sim = cosine_similarity(df_features)

In [20]:
# Step 4: Convert the similarity matrix into a DataFrame for easier analysis
cosine_sim_df = pd.DataFrame(cosine_sim, index=df.index, columns=df.index)

In [21]:
# Show the similarity matrix
print(cosine_sim_df)

          0         1         2         3         4         5         6   \
0   1.000000  0.971685  0.995436  0.999665  0.967240  0.997023  0.999848   
1   0.971685  1.000000  0.976485  0.971990  0.999320  0.977922  0.974169   
2   0.995436  0.976485  1.000000  0.993931  0.973398  0.998693  0.996028   
3   0.999665  0.971990  0.993931  1.000000  0.967690  0.996822  0.999709   
4   0.967240  0.999320  0.973398  0.967690  1.000000  0.975036  0.969886   
5   0.997023  0.977922  0.998693  0.996822  0.975036  1.000000  0.997862   
6   0.999848  0.974169  0.996028  0.999709  0.969886  0.997862  1.000000   
7   0.965614  0.999235  0.972177  0.966132  0.999949  0.973897  0.968362   
8   0.999665  0.971990  0.993931  1.000000  0.967690  0.996822  0.999709   
9   0.967240  0.999320  0.973398  0.967690  1.000000  0.975036  0.969886   
10  0.997023  0.977922  0.998693  0.996822  0.975036  1.000000  0.997862   
11  0.999848  0.974169  0.996028  0.999709  0.969886  0.997862  1.000000   
12  0.965614

In [22]:
def get_recommendations(user_id, top_n=3):
    # Get the similarity scores for the given user
    sim_scores = cosine_sim_df[user_id]
    
    # Sort users based on the similarity score (highest to lowest)
    similar_users = sim_scores.sort_values(ascending=False)[1:top_n+1]  # Skip the user themselves
    
    return similar_users

In [23]:
# Example: Recommend for user with ID 0
recommendations = get_recommendations(0)
print(recommendations)

40    1.000000
6     0.999848
28    0.999848
Name: 0, dtype: float64
